In [28]:
import pandas as pd
from scipy.io import arff
D_prime_data = arff.loadarff('C:/Users/udayk/OneDrive/Documents/AI projects/College Project on Bug prediction/NASADefectDataset-master/NASADefectDataset-master/CleanedData/MDP/D1/CM1.arff')
D_double_prime_data = arff.loadarff('C:/Users/udayk/OneDrive/Documents/AI projects/College Project on Bug prediction/NASADefectDataset-master/NASADefectDataset-master/CleanedData/MDP/D2/CM1.arff')
D_prime_data= pd.DataFrame(D_prime_data[0])
D_double_prime_data=pd.DataFrame(D_double_prime_data[0])
D_prime_data.head()



,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,6.0,9.0,2.0,1.0,0.0,16.0,5.0,0.20,8.0,2.0,...,14.0,0.16,19.0,44.0,15.0,15.0,32.0,4.00,25.0,b'N'
1,15.0,7.0,3.0,1.0,19.0,12.0,4.0,0.13,6.0,2.0,...,15.0,0.06,51.0,90.0,32.0,27.0,67.0,39.22,32.0,b'Y'
2,27.0,9.0,1.0,4.0,22.0,16.0,5.0,0.15,8.0,2.0,...,15.0,0.06,37.0,74.0,33.0,22.0,83.0,47.27,33.0,b'Y'
3,7.0,3.0,2.0,0.0,0.0,4.0,2.0,0.17,2.0,2.0,...,6.0,0.10,9.0,14.0,7.0,10.0,20.0,0.00,12.0,b'N'
4,51.0,25.0,13.0,0.0,14.0,48.0,13.0,0.12,24.0,2.0,...,85.0,0.08,192.0,229.0,71.0,20.0,172.0,11.67,106.0,b'N'


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Preprocess function
def preprocess_data(df):
    # Assuming the last column is the target variable
    X = df.iloc[:, :-1]  # Features
    y = df['Defective']    # Target variable (Y/N)
    #print(y)
    # Convert target variable to binary
    y = y.map({b'Y': 1, b'N': 0})
    
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preprocess_data(D_prime_data)
y_train


66     0
229    0
7      0
140    0
323    0
      ..
188    0
71     0
106    0
270    0
102    0
Name: Defective, Length: 275, dtype: int64

In [39]:
#!pip install imblearn 
from imblearn.over_sampling import SMOTE

# Handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_resampled, y_resampled)


RandomForestClassifier(random_state=42)

In [41]:
y_pred = model.predict(X_test)
print("Evaluation on D' Dataset:")
print(classification_report(y_test, y_pred))


Evaluation on D' Dataset:
              precision    recall  f1-score   support

           0       0.91      0.85      0.88        59
           1       0.36      0.50      0.42        10

    accuracy                           0.80        69
   macro avg       0.63      0.67      0.65        69
weighted avg       0.83      0.80      0.81        69



In [42]:
# Preprocess D'' dataset
X_new, _, y_actual, _ = preprocess_data(D_double_prime_data)

# Predict defects
predictions = model.predict(X_new)

# Map predictions back to Yes/No
#D_double_prime_data['Predicted_Defects'] = ['Yes' if pred == 1 else 'No' for pred in predictions]

# Compare with actual target variable
#y_actual = y_actual.map({b'Y': 1, b'N': 0})  # Adjust if target column position changes

# Evaluate predictions
print("Evaluation on D'' Dataset:")
print(classification_report(y_actual , predictions))

# Save predictions with actual results
D_double_prime_data.to_csv('predicted_defects_comparison.csv', index=False)


Evaluation on D'' Dataset:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       228
           1       0.69      0.82      0.75        33

    accuracy                           0.93       261
   macro avg       0.83      0.88      0.85       261
weighted avg       0.94      0.93      0.93       261

